In [32]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import keras
import math
import datetime as dt

In [25]:
df = pd.read_csv("D:/NLP/Volatility/Data/SP500/all_stocks_5yr.csv")
df.head()
# df.tail()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [27]:
# Get the list of company names
list_comp = df['Name'].unique()
num_comp = list_comp.shape[0]
print (num_comp, type(company_list))

505 <class 'numpy.ndarray'>


In [28]:
mask = df['Name'] == 'AAL'

In [30]:
df[mask]

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
1254,2018-02-01,54.00,54.64,53.59,53.88,3623078,AAL
1255,2018-02-02,53.49,53.99,52.03,52.10,5109361,AAL
1256,2018-02-05,51.99,52.39,49.75,49.76,6878284,AAL
1257,2018-02-06,49.32,51.50,48.79,51.18,6782480,AAL


In [31]:
df.loc[df['Name'] == "AAL"]

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
1254,2018-02-01,54.00,54.64,53.59,53.88,3623078,AAL
1255,2018-02-02,53.49,53.99,52.03,52.10,5109361,AAL
1256,2018-02-05,51.99,52.39,49.75,49.76,6878284,AAL
1257,2018-02-06,49.32,51.50,48.79,51.18,6782480,AAL


In [33]:
df

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [47]:
# Extract all data from year 2017
mask = df['date'].dt.year == int(2017)
include = df[mask]
exclude = df[~mask]
df_17 = df[mask]

In [52]:
df_17

,date,open,high,low,close,volume,Name
982,2017-01-03,47.28,47.340,46.135,46.30,6737752,AAL
983,2017-01-04,46.63,47.435,46.350,46.70,5859604,AAL
984,2017-01-05,46.52,46.930,45.610,45.89,6825316,AAL
985,2017-01-06,45.85,46.720,45.470,46.21,7260197,AAL
986,2017-01-09,46.01,47.340,45.780,47.08,4739142,AAL
...,...,...,...,...,...,...,...
619009,2017-12-22,72.30,72.370,71.790,71.99,1345683,ZTS
619010,2017-12-26,72.40,72.550,71.900,72.34,792134,ZTS
619011,2017-12-27,72.59,72.690,72.250,72.45,1159771,ZTS
619012,2017-12-28,72.49,72.600,72.140,72.39,710499,ZTS


In [51]:
df


,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL
...,...,...,...,...,...,...,...
619035,2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS
619036,2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
619037,2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
619038,2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS


In [53]:
lastdayfrom = pd.to_datetime('2017-01-03')
lastdayfrom

Timestamp('2017-01-03 00:00:00')

In [62]:
tau = 7 # This is a variable. Values to use are 3, 7, 15, and 30
# Create a mask for a specific stock
mask_stock = df['Name'] == 'AAL'
df_stock = df[mask_stock]
df_adj = df_stock[df_stock["date"] >= (pd.to_datetime('2017-01-03') - pd.Timedelta(days=tau))]

In [64]:
df_adj

,date,open,high,low,close,volume,Name
978,2016-12-27,48.29,48.92,48.140,48.61,2797847,AAL
979,2016-12-28,48.73,48.83,47.610,47.67,4153630,AAL
980,2016-12-29,47.80,48.00,46.960,47.25,3709262,AAL
981,2016-12-30,47.42,47.66,46.470,46.69,4495016,AAL
982,2017-01-03,47.28,47.34,46.135,46.30,6737752,AAL
...,...,...,...,...,...,...,...
1254,2018-02-01,54.00,54.64,53.590,53.88,3623078,AAL
1255,2018-02-02,53.49,53.99,52.030,52.10,5109361,AAL
1256,2018-02-05,51.99,52.39,49.750,49.76,6878284,AAL
1257,2018-02-06,49.32,51.50,48.790,51.18,6782480,AAL
